# Models

In [26]:
import pandas as pd
import numpy as np

PATH_FINAL = '../../data/final/'

In [27]:
students = pd.read_csv(PATH_FINAL+'students.csv', index_col=0)
doctors = pd.read_csv(PATH_FINAL+'doctors_all.csv', index_col=0)
nrpzs = pd.read_csv(PATH_FINAL+'nrpzs.csv', index_col=0)
demographics = pd.read_csv(PATH_FINAL+'demographics.csv', index_col=0)
insurances = pd.read_csv(PATH_FINAL+'insurances.csv', index_col=0)
dentists_age_estimate = pd.read_csv(PATH_FINAL+'dentists_age_estimate.csv', index_col=0)

/home/gary/Apps/miniconda3/envs/vzd/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: DtypeWarning: Columns (0,3,21,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gary/Apps/miniconda3/envs/vzd/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 1. Graduated students estimate 

In [66]:
students_start_count = students.reset_index().groupby(['date_start', 'university', 'degree']).count()[['index']].reset_index()
students_start_count = students_start_count[(2022 > students_start_count.date_start) & (students_start_count.date_start > 2015)]
total_sum = students_start_count.groupby(['date_start', 'university']).sum().reset_index()
total_sum.columns = ['date_start', 'university', 'total']
merged_students = pd.merge(students_start_count, total_sum)
merged_students['mudr_ratio'] = merged_students['index']/merged_students['total']
merged_students = merged_students[merged_students.degree =='MUDr.'][['date_start', 'university', 'mudr_ratio']]

In [72]:
uni_codes = {
    '11110': '1LFUK',
    '11120': '2LFUK',
    '11130': '3LFUK',
    '11140': 'PLUK',
    '11150': 'HKUK',
    '14110': 'MUNI',
    '15110': 'UPOL',
    '17110': 'OVA'
}

# uspesnosti https://www.ceskovdatech.cz/graphs/vs2.php
chance_to_graduate = {
    '1LFUK': .599,
    '2LFUK': .789,
    '3LFUK': .62,
    'PLUK': .758,
    'HKUK': .725,
    'MUNI': .648,
    'UPOL': .789,
    'OVA': .782 
}

new_students = dict()
graduate_estimate = dict()

all_tables = pd.DataFrame()
# https://dsia.msmt.cz//vystupy.html
for year in range(16, 22):
    table = pd.read_html(f'https://dsia.msmt.cz//vysluch{year}.html')[0]
    table.columns = table.iloc[0]
    table = table[1:][['rid','název školy/fakulty', 'zapsaných']].rename(columns={'zapsaných': 'count'})
    table = table[table['název školy/fakulty'].str.contains('ékařská')]
    table['university'] = table['rid'].apply(lambda x: uni_codes[x])
    table = table[['university', 'count']]
    table['date_start'] = 2000+year
    table['count'] = pd.to_numeric(table['count'])
    # table['graduate_estimate'] = round(table['zapsaných'].apply(int) * table['uni'].apply(lambda x: chance_to_graduate[x]))
    all_tables = all_tables.append(table)

In [86]:
graduates_estimate = pd.merge(all_tables, merged_students)
graduates_estimate['mudr'] = round(graduates_estimate['mudr_ratio']*graduates_estimate['count'])
graduates_estimate['mddr'] = round((1-graduates_estimate['mudr_ratio'])*graduates_estimate['count'])
graduates_estimate['mudr_graduate_estimate'] = round(graduates_estimate['mudr'].apply(int) * graduates_estimate['university'].apply(lambda x: chance_to_graduate[x]))
graduates_estimate['mddr_graduate_estimate'] = round(graduates_estimate['mddr'].apply(int) * graduates_estimate['university'].apply(lambda x: chance_to_graduate[x]))

mudr_estimates = graduates_estimate.groupby('date_start').sum()[['mudr_graduate_estimate']]
mddr_estimates = graduates_estimate.groupby('date_start').sum()[['mddr_graduate_estimate']]

mudr_dict = pd.Series(mudr_estimates.mudr_graduate_estimate.values,index=mudr_estimates.index+6).to_dict()
mddr_dict = pd.Series(mddr_estimates.mddr_graduate_estimate.values,index=mddr_estimates.index+5).to_dict()
mudr_dict, mddr_dict

({2022: 1619.0,
  2023: 1659.0,
  2024: 1621.0,
  2025: 1873.0,
  2026: 1690.0,
  2027: 2241.0},
 {2021: 200.0,
  2022: 229.0,
  2023: 206.0,
  2024: 194.0,
  2025: 197.0,
  2026: 181.0})

In [ ]:
graduate_estimate
# new_students

# TODO rozdeleni na zubare a nezubare podle skoly procentualne

In [ ]:
students[students.graduated == True].date_end.value_counts().sort_index()
# students[students.graduated == False].date_start.value_counts().sort_index()